<a href="https://colab.research.google.com/github/AvRob0/Avrob0.github.io/blob/main/RobinsonAveryFinalProjectEnd_To_End.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# imported librarys needed for End-to-end implementation of a Data Science Project
import pandas as pd
import numpy as np
# Loaded the dataset into the hotel_df variable
# and selected all the columns to see how many there are
hotel_df = pd.read_csv('hotel_bookings.csv')
hotel_df.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date'],
      dtype='object')

In [ ]:
hotel_df.info()
# Getting information about the dataset
# Getting the info of the dataset gives me the Range of the Index which is 119390
# It also gives me the total number of coloumns that are in the dataset and gives me all the columns names
# With their non-Null count and datatype

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

In [ ]:
# Getting the shape of the dataframe\dataset
# Which means on this dataset I have 36553 rows with 32 columns
hotel_df.shape

(119390, 32)

In [ ]:
# checking for null values
for i in hotel_df.columns:
    #print(i)
    #print(hotel_df[i].unique())
    if hotel_df[i].isna().sum() != 0:
      print('null values in', i, 'column', hotel_df[i].isna().sum())

null values in children column 4
null values in country column 488
null values in agent column 16340
null values in company column 112593


In [ ]:
# drop the company column since most of the values are 0
hotel_df = hotel_df.drop(columns = 'company')

# drops 4 rows all containing 0 value in the children column
hotel_df = hotel_df.dropna(subset=['children'])
#hotel_df
# filling the missing values in country column with the frequent mode value
hotel_df['country'].fillna(hotel_df['country'].mode()[0], inplace=True)

# then fill all the missing values in the agent column and use the value used for agent id.
hotel_df['agent'].fillna(0, inplace=True)


In [ ]:
# to replace TA\TO with undefined in distribution_channel column
hotel_df['distribution_channel'].replace('TA/TO', 'Undefined', inplace=True)

#replace Undefined, BB, FB, HB, SC to its meaning.
hotel_df['meal'].replace(['Undefined', 'BB', 'FB', 'HB','SC',], ['no meal', 'full board', 'half board', 'no meal', 'no meal'] , inplace=True)
#inplace = true -> directly modify the dataframe.


In [ ]:
# changing the children and agent column data type into a integer
hotel_df['children'].astype(int)
hotel_df['agent'].astype(int)

# turns the column data type into date and time
pd.to_datetime(hotel_df['reservation_status_date'])

0        2015-07-01
1        2015-07-01
2        2015-07-02
3        2015-07-02
4        2015-07-03
            ...    
119385   2017-09-06
119386   2017-09-07
119387   2017-09-07
119388   2017-09-07
119389   2017-09-07
Name: reservation_status_date, Length: 119386, dtype: datetime64[ns]

In [ ]:
hotel_df.duplicated().sum()
# We hae 5959 of duplicate rows
# Getting rid of the duplicates
hotel_df.drop_duplicates(inplace=True)

In [ ]:
#create total night column
hotel_df['total_nights'] = hotel_df['stays_in_weekend_nights'] + hotel_df['stays_in_week_nights']
#convert month name to number then create new arrival date column by combining year month date
hotel_df['arrival_date_month'] = pd.to_datetime(hotel_df['arrival_date_month'], format='%B').dt.month
hotel_df['date_arrival'] = pd.to_datetime(dict(year=hotel_df.arrival_date_year, month=hotel_df.arrival_date_month, day=hotel_df.arrival_date_day_of_month))

In [ ]:
# Now I am going to drop the unnecessary columns
drop_col = ['stays_in_weekend_nights','stays_in_week_nights', 'booking_changes','deposit_type','adr','arrival_date_year','arrival_date_month','arrival_date_day_of_month','arrival_date_week_number']

existing_columns = [col for col in drop_col if col in hotel_df.columns]

hotel_df.drop(columns=existing_columns, inplace=True)
# Then get a description of the dataframe to help with the statistical analysis
hotel_df.describe()


,is_canceled,lead_time,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,agent,days_in_waiting_list,required_car_parking_spaces,total_of_special_requests,total_nights,date_arrival
count,87365.000000,87365.000000,87365.000000,87365.000000,87365.000000,87365.000000,87365.000000,87365.000000,87365.000000,87365.000000,87365.000000,87365.000000,87365.000000,87365
mean,0.274938,79.914233,1.875946,0.138683,0.010828,0.039089,0.030424,0.184055,81.012041,0.748435,0.084256,0.698621,3.631271,2016-09-15 01:00:36.385280
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2015-07-01 00:00:00
25%,0.000000,11.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000,0.000000,0.000000,0.000000,2.000000,2016-04-01 00:00:00
50%,0.000000,49.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000,0.000000,0.000000,0.000000,3.000000,2016-09-20 00:00:00
75%,1.000000,125.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,234.000000,0.000000,0.000000,1.000000,5.000000,2017-04-01 00:00:00
max,1.000000,737.000000,55.000000,10.000000,10.000000,1.000000,26.000000,72.000000,535.000000,391.000000,8.000000,5.000000,69.000000,2017-08-31 00:00:00
std,0.446486,86.056876,0.626472,0.455945,0.113617,0.193808,0.369210,1.732197,109.940797,10.009095,0.281579,0.831953,2.763609,NaN
